In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets,models

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 300
BATCH_SIZE = 128

In [2]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        './.data',
        train = True,
        download = True,
        transform = transforms.Compose([
            # 과적합 방지를 위해 학습용 데이터 셋에 
            # RandomCrop과 RandomHorizontalFlip 같은 노이즈 추가 
            transforms.RandomCrop (32, padding = 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        './.data',
        train = False,
        download = True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])
    ),
    batch_size = BATCH_SIZE,
    shuffle = True
)


Files already downloaded and verified
Files already downloaded and verified


# CNN을 깊게 쌓는 방법 

   인공 신경망을 여러 개 겹친다고 학습 성능이 무한히 좋아지진 않는다. 
   여러 단계의 신경망을 거치며 최초 입력 이미지에 대한 정보가 소실되기 때문이다.
   
   ResNet은 이 문제를 해결하는 방안을 제시한다.
   ResNet의 핵심은 네트워크를 작은 블록인 Residual 블록으로 나누는 것이다.
   
   Residual 블록의 출력에 입력이었던 x를 더함으로써 모델을 훨씬 깊게 설계할 수 있다.
   입력과 출력의 관계를 바로 학습하기 보다 입력과 출력의 차이를 따로 학습하는 것이 성능이 좋다는 가설이다.
   
   신경망을 깊게 할수록 좋은 이유는 문제를 더 작은 단위로 분해하여 학습 효율이 좋아지기 때문이다.
   하지만 보통은 계층이 깊어짐에 따라 신호의 강도가 감소하는데,
   ResNet은 입력 데이터를 몇 계층씩 건너뛰어 출력에 더함으로써 이 현상을 완화한다. 
   
   
   ResNet은 층이 많아 복잡한 모델로 보일 수 있으나 Residual 블록을 반복적으로 쌓은 것 뿐이다.
   
   여기서, Residual 블록을 BasicBlock 이라는 새로운 파이토치 모듈을 정의해서 사용한다. 
   파이토치에서 nn.Modeule을 이용하여 모듈 위에 또 다른 모듈을 쌓아 올리 룻 있다.
   Conv2d 와 Linear 같은 모듈 모두 nn.Module 클래스를 상속한다. 
   
   

# Block 1 배치 정규화

https://youtu.be/nUUqwaxLnWs
 nn.BatchNorm2d 라는 새로운 계층이 있는데, 해당 계층에서는 배치 정규화가 수행된다.
 배치 정규화란 학습률을 너무 높게 잡으면 기울기가 소실되거나 발산하는 증상을 예방하여 학습 과정을 안정화하는 방법이다. 
 즉, 학습 중 각 계층에 들어가는 입력을 평균과 분산으로 정규화함으로써 학습을 효율적으로 만든다.
 이 계층은 자체적으로 정규화를 수행해 드롭아웃과 같은 효과를 내는 장점이 있다.
 
 드롭아웃은 학습 중 데이터 일부를 ㄹ배제하여 간접적으로 과적합을 막는 방식이지만, 배치 정규화는 신경망 내부 데이터에 
 직접 영향을 주는 방식이다.
 내

# Block 2 self.shortcut 모듈

self.shortcut 모듈은 in_palnes를  받아 self.bn2 계층의 출력 크기와 같은 planes와 더해준다. 
nn.Sequential 은 여러 모듈( nn.Module)을 하나의 모듈로 묶는 역할을 한다.
각 레이어를 데이터가 순차적으로 지나갈 떄 사용하면 코드를 간결하게 만들 수 있다.
계층과 활성화 함수를 정의해주면 순서대로 값을 전달해 처리한다.

# Block 3 BasicBlock 데이터 흐름 

입력 x가 들어와 컨볼루션, 배치 정규화, 활성화 함수를 거치게 하고 다시 입력 x를 self.short cut을 거치게 하여 
크기를 같게 하고 활성화 함수를 거친 값에 더한다.
그리고 이 값을 ReLU 활성화 함수를 통과시켜 최종 출력을 만든다. 

In [3]:
class BasicBlock(nn.Module):
    
    ## Blcok1 
    
    def __init__(self, in_planes, planes, stride = 1):
        super( BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size= 3,
                              stride = stride, padding = 1, 
                              bias = False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화 
        self.conv2 = nn.Conv2d(planes,planes, kernel_size=3,
                               stride = 1, padding =1, bias =False )
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화 
        
     ## Blcok2
    
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes : 
            self.shortcut = nn.Sequential (
                nn.Conv2d(in_planes, planes,
                         kernel_size=1, stride= stride,
                         bias = False),
                nn.BatchNorm2d(planes)
            )
    
     ## Blcok3 
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x))) # 컨볼루션 -> 배치정규화 -> 활성화 함수 
        out = self.bn2(self.conv2(out)) # 컨볼루션 -> 배치정규화 
        out += self.shortcut(x) # shortcut 
        out = F.relu(out)# 활성화 함수 
        
        return out
    

# Block 1 ResNet 모델 정의

이미지를 받아 컨볼루션과 배치 정규화 층을 거친 후 여러 BasicBlock 층을 통과하고 평균 풀링과 신경망을 거쳐 예측을 출력한다.
BasicBlock 클래스는 self._ make_layer() 모듈을 통해서 하나의 모듈로 객체화 되어 우리가 만들 ResNet 모델의 주요 층을 이룬다. 
self._ make_layer() 함수는 파이토치의 nn.Sequential 도구로 여러 BasicBlock 을 모듈 하나로 묶어주는 역할을 한다.
self._ make_layer() 함수가 반환한 self.layer1 과 같은 객체는 컨볼루션 계층과 마찬가지로 모듈(nn.Module) 로 취급하면 된다.
self,in_planes 변수는 self._ make_ layer( ) 함수에서 각 층을 만들 때 전 층의 채널 출력값을 기로가는 데 사용된다.
layer1 층이 입력받는 채널의 수가 16개 이기 때문에 16으로 초기화한다. 

각 층의 입력과 출력을 보면 첫 self.conv1 층은 3* 3의 커널 크기를 가지며 이미지가 갖고 있던 RGB 3색의 채널을 16개로 만든다.
layer1을 지나고 나면 똑같이 16 * 32 * 32 텐서를 갖게 되고, 
layer2 이후엔 32 * 16 * 16 텐서를 가지게 되고, 
layer3 를 거치곤 64 * 8 * 8 크기의 텐서를 갖게 된다.
마지막으로 평균 풀리으로 텐서에 있는 원소의 개수를 64개로 만든 후, 
64개의 입력을 받아 레이블 (10개) 마다 예측값을 내게 된다.


#  Block 2 self.make_layer()

이 함수는 맴버 변수인 self.in_planes 채널 개수로부터 직접 입력받은 인수인 planes 채널 개수만큼을 출력하는 BasicBlock을 생성한다.

layer1 : 16채널에서 16채널을 내보내는 BasicBlock 2개 

1ayer2 : 16채널을 받아 32채널을 출력하는  BasicBlock 1개 와 
         32채널에서 32채널을 내보내는  BasicBlock 1개 

layer3 : 32채널을 받아 64 채널을 출력하는  BasicBlock 1개 와 
         64채널에서 64채널을 출력하는 BasicBlock 1개 
         
여기서 주목해야 할 점은 16채널을 받아 32채널을 출력하는  BasicBlock과 
32채널을 받아 64 채널을 출력하는  BasicBlock 이ㅏㄷ.
이렇게 증폭하는 역할을 ㅏ하는 모듈들은 shortcut 모듈을 따로갖게 된다.
shortcut 모듈은 이전 입력을 중가층에 더해주어 이미지의 맥락이 보존될 수 있도록 하는 역할을 한다.


#  Block 3 ResNet 모델의 데이터 흐름 

ResNet 모델의 흐름은 다음과 같다.
입력이 들어오면 일반적인 방식과 같이 컨볼루션, 배치 정규화, 활성화 함수를 차례대로 통과하고, 
사전에 정의해둔 BasicBlock 층을 갖고 있는 layer1, layer2, 그리고 layer3을 통과하게 된다.
각 layer는 2개의 Residual 블록 갖고 있다.
이렇게 나온 값에 평균 풀링을 하고 마지막 계층을 거쳐 분류 결과를 출력합니다. 

In [4]:
## block1 
class ResNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_planes = 16
        
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,
                              stride=1, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.layer1 = self._make_layer(16,2,stride=1)
        self.layer2 = self._make_layer(32,2,stride=2)
        self.layer3 = self._make_layer(64,2,stride=2)
        
        self.linear = nn.Linear(64, num_classes)
     
    ## block2
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers=[]
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    ## block3
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x))) # CNN+ 배치 정규화 + ReLU
        
        out = self.layer1(out) # Basic Block + Basic Block 
        out = self.layer2(out) # ( Basic Block+shortcut ) + Basic Block 
        out = self.layer3(out) # ( Basic Block+shortcut ) + Basic Block 
        
        out = F.avg_pool2d(out, 8) # 평균 풀링 
        out = out.view(out.size(0),-1) # 신경망
        out = self.linear(out)  # 출력 
        return out

# 모델 사용법 

학습 효율을 높이기 위해 학습률 감소 기법을 사용한다.
학습률 감소는 학습이 진행하면서 최적화 학습률을 점점 낮춰서 더 정교하게 최적화한다.
파이토치 내부의 optim.lr_scheduler.StepLR 도구를 사용해 간단하게 적용할 수 있다.
Scheduler는 이폭마다 호출되며 step_size 를 50으로 지정하주어 50번 호출될 때
학슬률에 0.1(gamma 값)을 곱한다.
0.1로 시작한 학습률은 50 이폭 후에 0.01로 낮아진다


In [5]:
model =  ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.1 , momentum = 0.9, weight_decay = 0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 50, gamma = 0.1)

# 모델 출력 

In [6]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

# 학습 및 테스트용 코드


In [7]:
def train (model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0 
    correct = 0 
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            
            pred = output.max(1, keepdim = True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /=len(test_loader.dataset)
    test_accuracy = 100. * correct / len (test_loader.dataset)
    
    return test_loss, test_accuracy        

# 실행

scheduler.step() 함수로 학습률을 조금 낮춰주는 단계가 추가


< 발견한 오류>

C:\Users\sana\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:122: 

UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. 
In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  
Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. 

See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  
  

In [ ]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer, epoch)
    scheduler.step()
    test_loss, test_accuracy = evaluate( model, test_loader )
    
    
    print('[{}] Test Loss : {:.4f}, Accuracy: {:.2f}%'
          .format( epoch, test_loss, test_accuracy))

[1] Test Loss : 1.9522, Accuracy: 41.51%
[2] Test Loss : 1.1229, Accuracy: 60.50%
[3] Test Loss : 1.3881, Accuracy: 57.44%
[4] Test Loss : 0.8236, Accuracy: 71.75%
[5] Test Loss : 0.7985, Accuracy: 73.18%
[6] Test Loss : 0.7516, Accuracy: 74.53%
[7] Test Loss : 1.2869, Accuracy: 64.56%
[8] Test Loss : 0.6535, Accuracy: 77.63%
[9] Test Loss : 1.1696, Accuracy: 65.57%
[10] Test Loss : 0.9471, Accuracy: 68.02%
[11] Test Loss : 0.8376, Accuracy: 72.30%
[12] Test Loss : 0.9290, Accuracy: 71.82%
[13] Test Loss : 0.7394, Accuracy: 74.99%
[14] Test Loss : 0.7700, Accuracy: 75.78%
